# Descomposición en Valores Singulares (SVD)
## Implementación basada en las diapositivas del curso

En este notebook se implementa:
 1. Descomposición SVD de una matriz
 2. Aproximación de rango bajo
 3. Pseudoinversa de Moore-Penrose
 4. Resolución de problemas de mínimos cuadrados

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Configuración para mejor visualización
np.set_printoptions(precision=4, suppress=True)
plt.style.use('seaborn-v0_8-darkgrid')

### 1. Descomposición SVD Básica
 
Una matriz $A$ se puede escribir como suma de $r$ matrices de rango uno:

$$A = \sigma_1 u_1 v_1^t + \sigma_2 u_2 v_2^t + \ldots + \sigma_r u_r v_r^t$$

Donde $u_1, u_2, \ldots, u_r$ y $v_1, v_2, \ldots, v_r$ son las primeras columnas de $U$ y $V$.


In [2]:
def svd_basic(A, tol=1e-12):
    '''
    Calcula la descomposición en valores singulares (SVD) de una matriz A.
    
    A : array_like, forma (m, n)
            Matriz de entrada a descomponer.
    tol : float, opcional
            Tolerancia para considerar un valor singular como no nulo.
    Devuelve:
        U_r : ndarray, forma (m, r)
            Matriz de vectores singulares izquierdos.
        S : ndarray, forma (r,)
            Vector de valores singulares.
        Vt_r : ndarray, forma (r, n)
            Matriz de vectores singulares derechos transpuestos.
    '''
    A = np.asarray(A, dtype=float)
    m, n = A.shape

    # Descomposicion de autovalores de A^T A
    AtA = A.T @ A
    # np.linalg.eigh() devuelve autovalores y autovectores ordenados para matrices simétricas
    evals, V = np.linalg.eigh(AtA)

    # Tema 3 diapositiva 145: "Los autovalores de la matriz simétrica AtA son alpha^2_i
    # y los vectores singulares a derecha vi son los correspondientes autovectores ortonormales."

    # Ordenar autovalores en orden descendente
    idx = np.argsort(evals)[::-1]
    evals = evals[idx]
    V = V[:, idx]

    # Calcular valores singulares = sqrt(autovalores)
    singular_values = []
    V_r = []

    for lam, v in zip(evals, V.T):
        lam = max(lam, 0.0)
        sigma = np.sqrt(lam)
        if sigma > tol:
            singular_values.append(sigma)
            V_r.append(v)
        else:
            break

    if len(singular_values) == 0:
        return np.zeros((m, 0)), np.array([]), np.zeros((0, n))

    S = np.array(singular_values) # matriz de valores singulares en la diagonal
    V_r = np.stack(V_r, axis=1) # matriz de vectores singulares derechos
    r = len(S) # rango de la matriz se corresponde con el número de valores singulares no nulos

    # Despejamos U_r despejando desde A = U_r @ np.diag(S) @ V_r.T multiplicando por V_r en ambos lados
    # obteniendo U_r = A @ V_r.T / S
    U_r = np.zeros((m, r))
    for i in range(r):
        U_r[:, i] = (A @ V_r[:, i]) / S[i] 
    # También se podría obtener U_r a partir de esto:
    # Tema 3 diapositiva 145: "Los autovalores de la matriz simétrica AAt son alpha^2_i y m - n ceros, 
    # y los vectores singulares a la izquierda ui son los correspondientes autovectores ortonormales."

    Vt_r = V_r.T
    return U_r, S, Vt_r


Verificación comparando con Numpy SVD

In [3]:
np.random.seed(0)
A_test = np.random.randn(5, 3)

print("Matriz de prueba A:")
print(A_test)
print(f"\nDimensiones: {A_test.shape}")

U_my, S_my, Vt_my = svd_basic(A_test)
U_np, S_np, Vt_np = np.linalg.svd(A_test, full_matrices=False)

print("My singular values:     ", S_my)
print("NumPy singular values:  ", S_np)

A_my = U_my @ np.diag(S_my) @ Vt_my
A_np = U_np @ np.diag(S_np) @ Vt_np

print("\nReconstruction error (my SVD):   ", np.linalg.norm(A_test - A_my))
print("Reconstruction error (NumPy SVD):", np.linalg.norm(A_test - A_np))

Matriz de prueba A:
[[ 1.7641  0.4002  0.9787]
 [ 2.2409  1.8676 -0.9773]
 [ 0.9501 -0.1514 -0.1032]
 [ 0.4106  0.144   1.4543]
 [ 0.761   0.1217  0.4439]]

Dimensiones: (5, 3)
My singular values:      [3.5379 2.149  0.7688]
NumPy singular values:   [3.5379 2.149  0.7688]

Reconstruction error (my SVD):    1.2610473246791893e-15
Reconstruction error (NumPy SVD): 1.3316892628506722e-15


### 2. Aproximación de Rango Bajo

La aproximación de rango $k$ de una matriz se obtiene truncando la SVD:

$$A_k = \sum_{i=1}^{k} \sigma_i u_i v_i^T$$

Esta es la mejor aproximación de rango $k$ en el sentido de norma de Frobenius.

In [4]:
def low_rank_approximation(A, k):
    """Calcula la aproximación de rango k usando SVD"""
    U, S, Vt = svd_basic(A)
    
    # Truncar a los primeros k componentes
    U_k = U[:, :k]
    S_k = S[:k]
    Vt_k = Vt[:k, :]
    
    return U_k @ np.diag(S_k) @ Vt_k


# Prueba
np.random.seed(42)
A = np.random.randn(10, 8)

print("Matriz original A:")
print(f"Forma: {A.shape}")
print(f"Rango aproximado: {np.linalg.matrix_rank(A)}")

# Calcular SVD
U, S, Vt = svd_basic(A)
print(f"\nValores singulares: {S}")

# Aproximaciones de diferentes rangos
for k in [1, 2, 3, 5]:
    A_k = low_rank_approximation(A, k)
    error = np.linalg.norm(A - A_k, 'fro')
    print(f"\nRango {k}: error de Frobenius = {error:.4f}")


Matriz original A:
Forma: (10, 8)
Rango aproximado: 8

Valores singulares: [5.1031 4.2116 3.681  2.4449 2.2908 1.6601 1.4381 0.5234]

Rango 1: error de Frobenius = 6.9000

Rango 2: error de Frobenius = 5.4656

Rango 3: error de Frobenius = 4.0402

Rango 5: error de Frobenius = 2.2579


### 3. Pseudoinversa de Moore-Penrose
 
 La pseudoinversa se calcula como:
 
 $$A^+ = V \Sigma^+ U^T$$
 
 donde $\Sigma^+$ es la matriz diagonal con elementos $\frac{1}{\sigma_i}$ para $\sigma_i > 0$


In [5]:
def pseudo_inverse_svd(U, S, Vt, tol=1e-12):
    """Calcula la pseudoinversa usando SVD: A+ = V * S+ * U^T"""
    S_inv = np.array([1/s if s > tol else 0.0 for s in S])
    S_plus = np.diag(S_inv)
    # Vt tiene forma (r, n), necesitamos V de forma (n, r)
    V = Vt.T
    # A+ = V @ S+ @ U^T
    return V @ S_plus @ U.T


def pseudo_inverse(A, tol=1e-12):
    """Calcula la pseudoinversa de Moore-Penrose usando SVD"""
    U, S, Vt = svd_basic(A, tol=tol)
    return pseudo_inverse_svd(U, S, Vt, tol=tol)

### 4. Resolución de Problemas de Mínimos Cuadrados

Para resolver el problema de mínimos cuadrados:

$$\min_x \|Ax - b\|_2$$

la solución óptima se obtiene mediante la pseudoinversa:

$$x = A^+ b = V \Sigma^+ U^T b$$

Esta solución minimiza la norma euclidiana del residuo $r = Ax - b$.

In [6]:
A_ls = np.random.randn(6, 3)
b = np.random.randn(6)

A_plus = pseudo_inverse(A_ls)
x_ls = A_plus @ b

x_np, *_ = np.linalg.lstsq(A_ls, b, rcond=None)

print("My solution:  ", x_ls)
print("NumPy solution:", x_np)
print("Difference:", np.linalg.norm(x_ls - x_np))


My solution:   [ 0.1086 -0.752   0.3112]
NumPy solution: [ 0.1086 -0.752   0.3112]
Difference: 1.7867348644430382e-15


#### Propiedades de la solución:
- Si $A$ tiene rango completo por columnas, $x$ es único
- Si el sistema está sobredeterminado ($m > n$), $x$ es la solución de mínimos cuadrados
- Si el sistema está subdeterminado ($m < n$), $x$ es la solución de norma mínima

In [ ]:
# Verificación de propiedades de la pseudoinversa de Moore-Penrose
A = np.random.randn(4, 3)
A_plus = pseudo_inverse(A)

# Propiedades que debe cumplir:
# 1. A @ A+ @ A = A
# 2. A+ @ A @ A+ = A+
# 3. (A @ A+)^T = A @ A+
# 4. (A+ @ A)^T = A+ @ A

print("Propiedades de la pseudoinversa:")
print(f"1. ||A @ A+ @ A - A|| = {np.linalg.norm(A @ A_plus @ A - A):.2e}")
print(f"2. ||A+ @ A @ A+ - A+|| = {np.linalg.norm(A_plus @ A @ A_plus - A_plus):.2e}")
print(f"3. ||(A @ A+)^T - A @ A+|| = {np.linalg.norm((A @ A_plus).T - A @ A_plus):.2e}")
print(f"4. ||(A+ @ A)^T - A+ @ A|| = {np.linalg.norm((A_plus @ A).T - A_plus @ A):.2e}")

Propiedades de la pseudoinversa:
1. ||A @ A+ @ A - A|| = 1.92e-15
2. ||A+ @ A @ A+ - A+|| = 3.84e-15
3. ||(A @ A+)^T - A @ A+|| = 2.36e-16
4. ||(A+ @ A)^T - A+ @ A|| = 8.96e-16
